
# Machine Learning Foundation

## Curso 4, Parte a: K-Means Clustering LAB

__Propósito:__
O objetivo deste exercício de laboratório é aprender a usar um algoritmo de aprendizado não supervisionado, **K-means**, usando sklearn.

__Ao final deste laboratório você será capaz de:__
> 1. Execute um algoritmo K-means.
> 2. Entenda quais parâmetros são customizáveis ​​para o algoritmo.
> 3. Saber usar a curva de inércia para determinar o número ideal de clusters.

### Visão geral do K-Means

**K-means** é um dos algoritmos de agrupamento mais básicos. Ele se baseia em encontrar centros de cluster para agrupar pontos de dados com base na minimização da soma dos erros quadrados entre cada ponto de dados e seu centro de cluster.


In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

#lIBS
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.utils import shuffle


%config InlineBackend.figure_formats = ['retina']

In [ ]:
plt.rcParams['figure.figsize'] = [6,6]
sns.set_style("whitegrid")
sns.set_context("talk")

O agrupamento K-means é um dos algoritmos de agrupamento mais simples. Uma das limitações é que depende do ponto de partida dos clusters, e o número de clusters precisa ser definido previamente


### Pontos de partida do cluster
Vamos começar criando um conjunto de dados simples.

In [ ]:
# função auxiliar que nos permite exibir dados em 2 dimensões e destacar os clusters
def display_cluster(X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            plt.scatter(X[km.labels_==i,0],X[km.labels_==i,1],c = color[i],alpha = alpha,s=s)
            plt.scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)

In [ ]:
angle = np.linspace(0,2*np.pi,20, endpoint = False)
X = np.append([np.cos(angle)],[np.sin(angle)],0).transpose()
display_cluster(X)

In [ ]:
num_clusters = 2
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(X)
display_cluster(X,km,num_clusters)

In [ ]:
km = KMeans(n_clusters=num_clusters,random_state=20,n_init=1)
km.fit(X)
display_cluster(X,km,num_clusters)


## Pergunta:

Por que os clusters são diferentes quando executamos o K-means duas vezes?




É porque os pontos iniciais dos centros de cluster têm um impacto sobre onde estão os clusters finais. O ponto inicial dos clusters é controlado pelo estado aleatório.



### Determinando o número ideal de clusters

Vamos criar um novo conjunto de dados que consiste visualmente em alguns clusters e tentar agrupá-los

In [ ]:
n_samples = 1000
n_bins = 4  
centers = [(-3, -3), (0, 0), (3, 3), (6, 6)]
X, y = make_blobs(n_samples=n_samples, n_features=2, cluster_std=1.0,
                  centers=centers, shuffle=False, random_state=42)
display_cluster(X)

In [ ]:
num_clusters = 7
km = KMeans(n_clusters=num_clusters)
km.fit(X)
display_cluster(X,km,num_clusters)

Executando 4 clusteres


In [ ]:
num_clusters = 4
km = KMeans(n_clusters=num_clusters)
km.fit(X)
display_cluster(X,km,num_clusters)

- Neste caso, pode ser visualmente óbvio que quatro clusters é melhor que sete.
- Isso ocorre porque podemos visualizar facilmente os dados no espaço bidimensional.
- No entanto, os dados do mundo real geralmente têm mais de duas dimensões.
- Um conjunto de dados com um espaço dimensional superior é difícil de visualizar.
- Uma maneira de resolver isso é plotar a **inércia**

**inércia**: (soma do erro quadrático entre cada ponto e seu centro de cluster) em função do número de clusters.

In [ ]:
km.inertia_


### Problema 1:

Escreva um código que calcule a inércia para 1 a 10 clusters e plote a inércia como uma função do número de clusters.

In [2]:
inertia = []
list_num_clusters = list(range(1,11))
for num_clusters in list_num_clusters:
    km = KMeans(n_clusters=num_clusters)
    km.fit(X)
    inertia.append(km.inertia_)
    
plt.plot(list_num_clusters,inertia)
plt.scatter(list_num_clusters,inertia)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia');


NameError: name 'X' is not defined

### Clustering de cores de uma imagem


Os próximos exercícios usam uma imagem de pimentão. Vamos começar carregando:

In [ ]:
img = plt.imread('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%201/images/peppers.jpg', format='jpeg')
plt.imshow(img)
plt.axis('off')

In [ ]:
img.shape

A imagem acima tem 480 pixels de altura e 640 pixels de largura. Cada pixel possui 3 valores que representam quanto de vermelho, verde e azul ele possui. Abaixo você pode brincar com diferentes combinações de RGB para criar cores diferentes. No total, você pode criar $ 256 ^ 3 = 16.777.216 $ cores únicas.


In [ ]:
# atribui valores para o RGB. Cada valor deve estar entre 0 e 255R = 35
G = 95
B = 131
plt.imshow([[np.array([R,G,B]).astype('uint8')]])
plt.axis('off')


Primeiro vamos remodelar a imagem em uma tabela que tem um pixel por linha e cada coluna representa o canal vermelho, verde e azul.

In [ ]:
img_flat = img.reshape(-1, 3)
img_flat[:5,:]

In [ ]:
img_flat.shape

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0).fit(img_flat)

In [ ]:
img_flat2 = img_flat.copy()

# loops for each cluster center
for i in np.unique(kmeans.labels_):
    img_flat2[kmeans.labels_==i,:] = kmeans.cluster_centers_[i]

In [ ]:
img2 = img_flat2.reshape(img.shape)
plt.imshow(img2)
plt.axis('off');

### Problema 2:
Escreva uma função que receba a imagem e o número de clusters (k) e retorne (1) a imagem quantizada em k cores e (2) a inércia

In [ ]:
def image_cluster(img, k):
    img_flat = img.reshape(img.shape[0]*img.shape[1],3)
    kmeans = KMeans(n_clusters=k, random_state=0).fit(img_flat)
    img_flat2 = img_flat.copy()

    # loops para cada centro de cluster
    for i in np.unique(kmeans.labels_):
        img_flat2[kmeans.labels_==i,:] = kmeans.cluster_centers_[i]
        
    img2 = img_flat2.reshape(img.shape)
    return img2, kmeans.inertia_


### Problema 3:

Chame a função para k entre 2 e 20 e desenhe uma curva de inércia. Qual é o número ideal de clusters?

In [ ]:
k_vals = list(range(2,21,2))
img_list = []
inertia = []
for k in k_vals:
    
    img2, ine = image_cluster(img,k)
    img_list.append(img2)
    inertia.append(ine)  

In [ ]:
# Plotar para encontrar o número ideal de clusters
plt.plot(k_vals,inertia)
plt.scatter(k_vals,inertia)
plt.xlabel('k')
plt.ylabel('Inertia');

### Problema 4:
Plote em uma grade todas as imagens para os diferentes valores de k.

In [ ]:
plt.figure(figsize=[10,20])
for i in range(len(k_vals)):
    plt.subplot(5,2,i+1)
    plt.imshow(img_list[i])
    plt.title('k = '+ str(k_vals[i]))
    plt.axis('off');